In [1]:
import pandas as pd
import numpy as np
import statsmodels.api as sm
from scipy.optimize import minimize

In [2]:
zhongzhai = pd.read_excel('久期计算.xlsx', sheet_name='中债新综合')
lilvzhaiji = pd.read_excel('久期计算.xlsx', sheet_name='基金')

In [3]:
zhongzhai.dropna(inplace=True)
zhongzhai.reset_index(inplace=True,drop=True)
lilvzhaiji.dropna(inplace=True)
lilvzhaiji.reset_index(inplace=True,drop=True)

In [4]:
zhongzhai_column = zhongzhai.columns
lilvzhaiji_column = lilvzhaiji.columns

In [5]:
merged_df = pd.merge(zhongzhai, lilvzhaiji, on='date').reset_index(drop=True)

### zhongzhai column 6 + lilvzhaiji column 39 +date column = 46 cols

In [7]:
zhongzhai = merged_df[zhongzhai_column]
lilvzhaiji = merged_df[lilvzhaiji_column]

In [8]:
zhongzhai = zhongzhai.loc[0:39]
lilvzhaiji = lilvzhaiji.loc[0:39]

In [9]:
new_zhongzhai = zhongzhai.drop(columns='date')
new_lilvzhaiji = lilvzhaiji.drop(columns='date')

In [10]:
new_zhongzhai

,Y1,Y1-3,Y3-5,Y5-7,Y7-10,Y10
0,0.000298,-0.012452,-0.041363,-0.063276,-0.087458,-0.253737
1,0.004113,0.013483,0.015373,0.021778,0.032249,-0.020316
2,0.009837,0.031846,0.06326,0.076635,0.094058,0.144669
3,0.007095,0.004777,0.040461,0.073779,0.065698,0.105366
4,0.012045,-0.009085,-0.029818,-0.094712,-0.141401,-0.4486
5,0.00817,0.015737,0.027665,0.031791,0.041621,0.160815
6,0.010795,0.023424,0.045582,0.079067,0.102978,0.155285
7,0.009483,0.019586,0.029849,0.039391,0.039571,0.042273
8,0.015331,0.025591,0.055229,0.034361,0.097461,0.145426
9,-0.002923,0.014063,0.029355,0.000473,0.046895,0.075175


In [11]:
new_zhongzhai = new_zhongzhai.astype(float)
new_lilvzhaiji = new_lilvzhaiji.astype(float)

In [12]:
def objective_function(params, X, Y):
    return np.sum((Y - X @ params) ** 2)

# 定义约束条件
def constraint_sum_upper(params):
    return 1.4 - np.sum(params)

def constraint_sum_lower(params):
    return np.sum(params) - 0.8

constraints = [{'type': 'ineq', 'fun': lambda params: params},  # beta > 0
               {'type': 'ineq', 'fun': lambda params: 1 - params},  # beta < 1
               {'type': 'ineq', 'fun': constraint_sum_upper},  # sum(beta) < 1.4
               {'type': 'ineq', 'fun': constraint_sum_lower}]  # sum(beta) > 0.8

# 创建一个新的 DataFrame 用于存储结果
adjusted_coeffs_df = pd.DataFrame(index=new_zhongzhai.columns)

# 对 new_lilvzhaiji 的每一列进行回归
for column in new_lilvzhaiji.columns:
    Y = new_lilvzhaiji[column]
    X = new_zhongzhai

    # 初始参数
    initial_params = np.full(X.shape[1], 0.1)  # 初始化为0.1，确保所有系数在合理范围内
    
    # 进行优化
    result = minimize(objective_function, initial_params, args=(X, Y), constraints=constraints)
    
    if result.success:
        optimized_params = result.x
        total_beta = np.sum(optimized_params[1:])
        
        # 根据整体仓位调整测算久期
        if total_beta > 1:
            scaling_factor = 1 / total_beta
            adjusted_params = optimized_params * scaling_factor
        else:
            adjusted_params = optimized_params
        
        # 存储结果（忽略常数项的系数）
        adjusted_coeffs_df[column] = adjusted_params
    else:
        print(f'Optimization failed for {column}. Reason:', result.message)

# 转置结果 DataFrame
transposed_adjusted_coeffs_df = adjusted_coeffs_df.T

# 打印结果 DataFrame
print(transposed_adjusted_coeffs_df)


                     Y1          Y1-3          Y3-5          Y5-7  \
100051.OF  8.000000e-01 -5.828671e-16  2.697071e-16  2.628275e-16   
320021.OF  8.000000e-01  3.191891e-16 -1.867387e-16  2.267204e-16   
006031.OF  8.000000e-01 -1.232087e-15  9.116470e-16  1.024523e-15   
005461.OF  8.000000e-01 -4.996004e-16 -1.619652e-15 -4.330563e-16   
040023.OF  8.000000e-01  6.106227e-16 -4.829337e-16 -4.710242e-17   
110035.OF  8.000000e-01  1.387779e-16  8.539982e-17 -7.621712e-17   
002405.OF  1.000000e+00  9.406715e-02 -4.175045e-16  1.300797e-16   
006932.OF  8.308435e-17  6.382648e-01  2.547770e-16  1.617352e-01   
159649.OF -3.731987e-18  3.545063e-01  1.626508e-01  2.511105e-01   
511520.OF  1.040402e-17  1.839212e-17  3.072895e-17  3.109517e-01   
006172.OF  7.162342e-02  6.609776e-02  1.652660e-01  8.368756e-18   
009786.OF -1.990742e-17  3.726922e-01  1.644859e-01  2.804702e-01   
016108.OF  1.926389e-17  4.689469e-01  2.203415e-01  2.843649e-01   
013857.OF -1.215724e-17  3.262158e

In [13]:
transposed_adjusted_coeffs_df['duration'] = 0.4597*transposed_adjusted_coeffs_df['Y1']+1.7976*transposed_adjusted_coeffs_df['Y1-3']+3.5298*transposed_adjusted_coeffs_df['Y3-5']+5.2875*transposed_adjusted_coeffs_df['Y5-7']+7.2852*transposed_adjusted_coeffs_df['Y7-10']+14.8624*transposed_adjusted_coeffs_df['Y10']

In [14]:
transposed_adjusted_coeffs_df['duration']

100051.OF    0.367760
320021.OF    0.367760
006031.OF    0.367760
005461.OF    0.367760
040023.OF    0.367760
110035.OF    0.367760
002405.OF    0.628795
006932.OF    2.002520
159649.OF    2.977005
511520.OF    7.891405
006172.OF    7.371544
009786.OF    4.539523
016108.OF    3.493360
013857.OF    3.312388
013653.OF    4.808458
012273.OF    4.146582
008030.OF    4.751703
018642.OF    2.833498
006206.OF    4.660454
002549.OF    1.049482
013236.OF    0.819005
003327.OF    7.431699
003949.OF    1.142691
004200.OF    2.451167
519782.OF    2.062563
217022.OF    1.737088
000191.OF    2.085483
050027.OF    4.023793
006337.OF    2.914956
014785.OF    3.005612
040040.OF    3.164742
000914.OF    2.602186
000186.OF    3.542498
004089.OF    3.113183
270048.OF    2.766986
000032.OF    3.997740
004042.OF    5.619006
400030.OF    3.203945
007214.OF    9.341402
Name: duration, dtype: float64

In [15]:
transposed_adjusted_coeffs_df['duration'].to_excel('久期结果.xlsx')